In [1]:
from urllib.request import urlopen
import urllib.request as req
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd
import numpy as np
from urllib.request import urlopen #url의 html 을 가져 오기 위한 패키지
from bs4 import BeautifulSoup  #크롤링 필수 패키지 설치하려면 cmd창에서 pip install bs4
import os
import re
from selenium import webdriver
from bs4 import BeautifulSoup #크롤링 도구
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import os
import re
import time
import json
import datetime
from konlpy.tag import Okt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pylab as pl
import statsmodels.api as sm
from sklearn.cluster import DBSCAN
import sklearn as skl
import sklearn.model_selection
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import random
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from ckonlpy.tag import Twitter
import string
import glob
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import warnings
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram
from matplotlib import font_manager, rc
from ckonlpy.tag import Postprocessor



In [2]:
#데이터 불러오기
try:
    data = pd.read_excel('C:/Users/USER/Desktop/통계용어사업data.xlsx')
    data['words'] = [i.replace("'", "").replace('[', '').replace(']', '').replace(' ', '').split(',') for i in
                    list(data['words'])]  # words가 텍스트 형식으로 되어 있을 경우
except:
    #2018, 19, 20 일부 뉴스 가져와서 합치기
    #2018년 뉴스
    data2018= pd.read_excel('C:/Users/USER/Desktop/2018newsdata1105.xlsx', index_col=0)

    #2019년 뉴스
    data2019 = pd.read_excel('C:/Users/USER/Desktop/data1015.xlsx', index_col=0)

    #2020년 뉴스
    data2020 = pd.read_excel('C:/Users/USER/Desktop/1~8newsurl/data0928.xlsx', index_col=0)

    #통합 데이터 만들기
    colnames = ['date','title','text','category','words']
    data2018['date'] = [datetime.datetime.strptime(str(i), '%Y%m%d') for i in data2018['date']] #date변수 date타입으로 바꿔주기
    data = pd.concat([data2018[colnames],data2019[colnames],data2020[colnames]])
    data.to_excel('C:/Users/USER/Desktop/통계용어사업data.xlsx', index= False)
    del data2018; del data2019; del data2020;

In [3]:
print(len(data))
print(type(data['words'][0]))

141365
<class 'list'>


In [4]:
embedding_model = Word2Vec(data['words'], size=100, window = 5, min_count=5, workers=4, iter=100)
embedding_model.save('C:/Users/USER/Desktop/고용단어분류_w2v_min5.model') # 모델 저장

embedding_model2 = Word2Vec(data['words'], size=100, window = 5, min_count=50, workers=4, iter=100)
embedding_model2.save('C:/Users/USER/Desktop/고용단어분류_w2v_min50.model') # 모델 저장

In [2]:
#####################텍스트 분석 언더바 해서 
data2018 = pd.read_excel('C:/Users/USER/Desktop/통계용어사업data2018.xlsx')

#단어 데이터 불러오기
work_word_data = pd.read_excel('C:/Users/USER/Desktop/고용관련단어.xlsx')
#단어 리스트 만들기
work_word_list = [i for i in work_word_data['선정용어']]
work_word_list

#밑줄 단어 만들기
work_word_list2 = []
for i in  work_word_list:
    work_word_list2.append(i.replace(' ','_'))

#띄어쓰기 없는단어 만들기
work_word_list3 = []
for i in  work_word_list:
    work_word_list3.append(i.replace(' ',''))

In [3]:

#단어 리스트 형태소 분석기에 넣기
twi = Twitter()
for i in range(len(work_word_list2)):
    twi.add_dictionary(work_word_list2[i],'Noun')
    twi.add_dictionary(work_word_list[i],'Noun')
    twi.add_dictionary(work_word_list3[i],'Noun')

#통합어 딕셔너리 만들기
replace = {}
for i in range(len(work_word_list2)):
    replace[work_word_list2[i]] = work_word_list[i]
    replace[work_word_list3[i]] = work_word_list[i]

#분석할 데이터 셋 만들기 2018년도만 가져오기
postprocessor = Postprocessor(base_tagger=twi, replace=replace, passtags={'Noun'})
under_bar_text = [m.replace(' ','_') for m in data2018['text']]
#형태소 분석 하기
words = [[j[i][0] for i in range(len(j))] for j in [postprocessor.pos(i) for i in tqdm(under_bar_text)]]

#배제어 등록하기
Prohibit_words = ['기자','연합뉴스','뉴시스','시사저널','신문','뉴스','사진','헤럴드경제','노컷뉴스','파이낸셜뉴스','특파원',
                  '라며','대해','지난','위해','오전','오후','무단','배포','이데일리','머니투데이','앵커','지금','때문','이번',
                  '통해','정도','경우','관련','이미지','출처','일보','바로가기','까지','여개','도록','이나','재배포','처럼','면서',
                  '거나','이제','지난달','어요']

#배제어 제거, 한 글자 제거하기
j = 0
for i in tqdm(words):
    for k in Prohibit_words:
        while k in i:
            i.remove(k)
    words[j] = i
    j += 1 #불용어 제외

for k in range(len(words)):
    words[k] = [i for i in words[k] if len(i) > 1]  # 한글자 제외

data2018['words2'] = words
data2018.to_excel('C:/Users/USER/Desktop/통계용어사업data2018_2.0.xlsx', index=False)

C:\Users\USER\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
100%|██████████████████████████████████████████████████████████████████████████| 23930/23930 [00:06<00:00, 3844.40it/s]


In [6]:
a = []
for i in work_word_list:
    k= 0
    for j in data2018['words2']:
        if i in j:
            k += 1
    a.append(k)
a

[1,
 2,
 192,
 435,
 618,
 130,
 38,
 0,
 1,
 2,
 11,
 459,
 81,
 22,
 2346,
 3,
 138,
 9,
 2,
 0,
 1115,
 754,
 144,
 1311,
 26,
 2,
 90,
 0,
 0,
 122,
 0,
 0,
 1905,
 1,
 0,
 7,
 10,
 1507,
 0,
 1,
 21,
 42,
 33,
 61,
 0,
 2,
 1476,
 40,
 199,
 1,
 1,
 2282,
 1,
 17,
 3,
 2,
 0,
 5,
 53,
 2,
 0,
 266,
 88,
 106,
 0,
 0,
 12,
 6,
 12,
 48,
 1,
 1,
 12,
 5,
 11,
 4079,
 11,
 0,
 41,
 0,
 0,
 39,
 9,
 27,
 24,
 0,
 0,
 152,
 0,
 532,
 256,
 8,
 5026,
 327,
 0,
 0,
 124,
 0,
 416,
 35,
 219,
 3,
 0,
 33,
 0,
 6,
 2,
 0,
 0,
 8,
 39,
 0,
 1,
 0,
 0,
 2,
 0,
 6,
 6,
 5308,
 1]

In [7]:
pd.DataFrame(a).to_excel('C:/Users/USER/Desktop/8.xlsx', index=False)
